<a href="https://colab.research.google.com/github/krazyjoy/Hackathon/blob/main/104.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re
import time
import os
from IPython.display import clear_output


In [3]:
headers = {'User-Agent':'GoogleBot'}

In [4]:
url = 'https://static.104.com.tw/category-tool/json/Area.json'
resp = requests.get(url)
df1 = []
for i in resp.json()[0]['n']:
    ndf = pd.DataFrame(i['n'])
    ndf['city'] = i['des']
    df1.append(ndf)
df1=pd.concat(df1, ignore_index=True)
df1 = df1.loc[:,['city','des','no']]
df1 = df1.sort_values('no')
df1

,city,des,no
0,台北市,台北市中正區,6001001001
1,台北市,台北市大同區,6001001002
2,台北市,台北市中山區,6001001003
3,台北市,台北市松山區,6001001004
4,台北市,台北市大安區,6001001005
...,...,...,...
360,金門縣,金門縣烏坵鄉,6001022006
361,連江縣,連江縣南竿鄉,6001023001
362,連江縣,連江縣北竿鄉,6001023002
363,連江縣,連江縣莒光鄉,6001023003


In [5]:
url= 'https://static.104.com.tw/category-tool/json/JobCat.json'
resp = requests.get(url)
df2 = []
for i in resp.json():
    for j in i['n']:
        ndf = pd.DataFrame(j['n'])
        ndf['des1'] = i['des']# 職務大分類
        ndf['des2'] = j['des']# 職務小分類
        df2.append(ndf)
df2 = pd.concat(df2, ignore_index=True)
df2 = df2.loc[:,['des1', 'des2', 'des', 'no']]
df2 = df2.sort_values('no')
df2

,des1,des2,des,no
1,經營╱人資類,經營╱幕僚類人員,經營管理主管,2001001001
0,經營╱人資類,經營╱幕僚類人員,儲備幹部,2001001002
2,經營╱人資類,經營╱幕僚類人員,主管特別助理,2001001003
3,經營╱人資類,人力資源類人員,人力資源主管,2001002001
6,經營╱人資類,人力資源類人員,人力資源人員,2001002002
...,...,...,...,...
464,其他職類,其他類人員,加油員,2018002010
465,其他職類,其他類人員,派報生╱傳單派送,2018002011
466,其他職類,其他類人員,清潔工╱資源回收人員,2018002012
467,其他職類,其他類人員,家事服務人員,2018002013


In [7]:
tmp = pd.DataFrame([re.sub('\.pkl','',file)for file in os.listdir('./data')],columns=['no'])
df1 = pd.merge(df1, tmp, how='left',on='no',indicator=True)
df1 = df1.loc[df1['_merge']!='both',:]
df1

,city,des,no,_merge
0,台北市,台北市中正區,6001001001,left_only
1,台北市,台北市大同區,6001001002,left_only
2,台北市,台北市中山區,6001001003,left_only
3,台北市,台北市松山區,6001001004,left_only
4,台北市,台北市大安區,6001001005,left_only
...,...,...,...,...
360,金門縣,金門縣烏坵鄉,6001022006,left_only
361,連江縣,連江縣南竿鄉,6001023001,left_only
362,連江縣,連江縣北竿鄉,6001023002,left_only
363,連江縣,連江縣莒光鄉,6001023003,left_only


In [ ]:
columns = ['公司名稱','公司編號','公司類別','公司類別描述', '公司連結','職缺名稱','職務性質','職缺大分類', '職缺中分類','職缺小分類', '職缺編號', '職務內容','更新日期', '職缺連結', '標籤','公司地址','地區','經歷','學歷']

for areades, areacode in zip(df1['des'],df1['no']):
    values = []
    for jobdes1, jobdes2, jobdes, jobcode in zip(df2['des1'], df2['des2'], df2['des'], df2['no']):
        print(areades, ' | ', jobdes1, ' - ', jobdes2, ' - ' ,jobdes)
        page = 1
        while page <150:
            try:
                url = 'https://www.104.com.tw/jobs/search/?ro=0&jobcat={}&jobcatExpansionType=1&area={}&order=11&asc=0&page={}&mode=s&jobsource=2018indexpoc'.format(jobcode, areacode, page)
                print(url)
                resp = requests.get(url,headers=headers)
                soup = BeautifulSoup(resp.text)
                soup2 = soup.find('div',{'id':'js-job-content'}).findAll('article',{'class':'b-block--top-bord job-list-item b-clearfix js-job-item'})
                print(len(soup2))

                for job in soup2:
                                        
                    update_date = job.find('span',{'class':'b-tit__date'}).text
                    update_date = re.sub('\r|\n| ','',update_date)

                    try:
                        address = job.select('ul > li > a')[0]['title']
                        address = re.findall('公司住址：(.*?)$',address)[0]
                    except:
                        address = ''
                   
                    loc = job.find('ul',{'class':'b-list-inline b-clearfix job-list-intro b-content'}).findAll('li')[0].text
                    exp = job.find('ul',{'class':'b-list-inline b-clearfix job-list-intro b-content'}).findAll('li')[1].text
                    try:
                        edu = job.find('ul',{'class':'b-list-inline b-clearfix job-list-intro b-content'}).findAll('li')[2].text
                    except:
                        edu = ''
                    
                    try:
                        content = job.find('p').text
                    except:
                        content = ''
                    try:
                        tags = [tag.text for tag in soup2[0].find('div',{'class':'job-list-tag b-content'}).findAll('span')]
                    except:
                        tags = []
                    
                    
                    value = [job['data-cust-name'], # 公司名稱
                             job['data-cust-no'], # 公司編號
                             job['data-indcat'], # 公司類別
                             job['data-indcat-desc'], # 公司類別描述
                             job.select('ul > li > a')[0]['href'], # 公司連結
                             job['data-job-name'],# 職缺名稱
                             job['data-job-ro'], # 職務性質 _判斷全職兼職 1全職/2兼職/3高階/4派遣/5接案/6家教
                             jobdes1, # 職缺大分類
                             jobdes2, # 職缺中分類
                             jobdes, # 職缺小分類
                             job['data-job-no'],# 職缺編號
                             content, # 職務內容
                             update_date, # 更新日期
                             job.find('a',{'class':'js-job-link'})['href'], # 職缺連結
                             tags, # 標籤
                             address,# 公司地址
                             loc, # 地區
                             exp,# 經歷
                             edu  # 學歷
                            ]
                    values.append(value)
                
                page+=1
                print(len(values))
                if len(soup2) < 20:
                    break
            except:
                print('Retry')
        
    df = pd.DataFrame()
    df = pd.DataFrame(values, columns=columns)
    df.to_pickle('./data/' + areacode + '.pkl')
    clear_output()
    !pip install xlsxwriter
    df = []
    for i in os.listdir('./data/'):
    ndf = pd.read_pickle('./data/' + i)
    df.append(ndf)
    df = pd.concat(df, ignore_index=True)
    df.info()
    filePath = 'C:/Users/USER/Desktop/104爬蟲/example.xlsx' #assign filename and file path
    df.to_excel("output.xlsx", engine = 'xlsxwriter') 
    print('===================================  Save Data  ===================================')


台北市中正區  |  經營╱人資類  -  經營╱幕僚類人員  -  經營管理主管
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2001001001&jobcatExpansionType=1&area=6001001001&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
20
20
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2001001001&jobcatExpansionType=1&area=6001001001&order=11&asc=0&page=2&mode=s&jobsource=2018indexpoc
20
40
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2001001001&jobcatExpansionType=1&area=6001001001&order=11&asc=0&page=3&mode=s&jobsource=2018indexpoc
20
60
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2001001001&jobcatExpansionType=1&area=6001001001&order=11&asc=0&page=4&mode=s&jobsource=2018indexpoc
16
76
台北市中正區  |  經營╱人資類  -  經營╱幕僚類人員  -  儲備幹部
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2001001002&jobcatExpansionType=1&area=6001001001&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
20
96
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2001001002&jobcatExpansionType=1&area=6001001001&order=11&asc=0&page=2&mode=s&jobsource=2018ind

In [ ]:
df = []
for i in os.listdir('./data/'):
    ndf = pd.read_pickle('./data/' + i)
    df.append(ndf)
df = pd.concat(df, ignore_index=True)
df.info()